In [78]:
import numpy as np
import pandas as pd

import keras
from keras.preprocessing.sequence import TimeseriesGenerator

import matplotlib.pyplot as plt

In [10]:
dim=(2,2,2)

In [11]:
batch_size = 1
n_channels =1

In [5]:
train = pd.read_csv("gs://123test_bucket/train.csv")

In [12]:
np.empty((batch_size, *dim, n_channels))

array([[[[[0.00000000e+000],
          [4.67846830e-310]],

         [[4.67846830e-310],
          [4.67846830e-310]]],


        [[[4.67846830e-310],
          [6.93022916e-310]],

         [[6.93022916e-310],
          [6.93022916e-310]]]]])

In [30]:
data = np.array([[i] for i in range(50)])
targets = np.array([[i] for i in range(50)])


In [119]:
length = len(data) - 10

50

In [126]:
data_gen = TimeseriesGenerator(data, targets,
                               length=4, sampling_rate=1,
                               batch_size=3)

In [133]:
#assert len(data_gen) == 20
print(len(data_gen))

16


In [131]:
batch_0 = data_gen[0]
x, y = batch_0

In [135]:
len(x) * len(data_gen)

48

In [39]:
assert np.array_equal(x,
                      np.array([[[0], [2], [4], [6], [8]],
                                [[1], [3], [5], [7], [9]]]))
assert np.array_equal(y,
                      np.array([[10], [11]]))

In [114]:
len(train[['building_id', 'meter']].drop_duplicates())

2380

In [74]:
adf = train.loc[train['building_id'] == 1, 'meter_reading']

length = 168
batch_size = len(adf) - length

data_gen = TimeseriesGenerator(adf.values,adf.values,length=168,batch_size = batch_size)

In [75]:
batch_0 = data_gen[0]
x, y = batch_0

In [86]:
train_sub = train[train.building_id.isin(np.arange(0,10))]
train_sub.describe()

In [137]:
# https://stackoverflow.com/questions/55116638/use-keras-timeseriesgenerator-function-to-generate-squence-group-by-some-id/55118459#55118459
# https://keras.io/preprocessing/sequence/
class DataGenerator(keras.utils.Sequence):
    def __init__(self, dt, length = 168, batch_size = batch_size):
        # length represents look back, i.e. how many time steps prior we need.
        self.tgs = list()
        for i in range(dt['building_id'].min(),dt['building_id'].max()+1):
            sub = dt.loc[dt['building_id'] == i, ['meter', 'meter_reading']]
            for meter in sub['meter'].unique():
                adf = sub.loc[sub['meter'] == meter, 'meter_reading']
                # reformatted time series needs to be generated in batches
                self.tgs.append(TimeseriesGenerator(adf.values,adf.values,length,batch_size=batch_size))
        self.len = sum([len(tg) for tg in self.tgs])
        self.idx_i = list()
        self.idx_j = list()

        for i, tg in enumerate(self.tgs):
            self.idx_i.extend(list(range(len(tg))))
            self.idx_j.extend([i]*len(tg))    
        #print ( self.idx_i,  self.idx_j)

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        return self.tgs[self.idx_j[index]][self.idx_i[index]]



In [160]:
assert len(train_sub[['building_id', 'meter']].drop_duplicates()) == 12
df_category = len(train_sub[['building_id', 'meter']].drop_duplicates())
df_category

12

In [142]:
# Test
length = 168
batch_size = 3
g = DataGenerator(train_sub,length, batch_size = batch_size)

In [163]:
df_len = len(train_sub)
df_len 

102372

In [167]:
assert round((df_len - len(g)*batch_size + (length*df_category))/df_len) == 0

In [175]:
x, y = g[-1000] # For example

In [177]:
y

array([928.628, 970.838, 928.628])

In [176]:
x

array([[1013.05 ,  970.838,  970.838,  970.838,  928.628, 1224.1  ,
         970.838, 1519.57 , 1899.47 , 1688.41 , 1519.57 , 1519.57 ,
        1435.15 , 1561.78 , 1308.52 , 1181.89 , 1055.26 , 1139.68 ,
         759.786,  886.418, 1224.1  , 1097.47 , 1139.68 , 1013.05 ,
         759.786,  970.838,  928.628,  886.418,  928.628, 1139.68 ,
        1308.52 , 1350.73 , 1435.15 , 1519.57 , 1435.15 , 1392.94 ,
        1392.94 , 1477.36 , 1477.36 , 1435.15 , 1139.68 , 1097.47 ,
         844.207, 1013.05 ,  759.786, 1055.26 , 1013.05 ,  970.838,
         844.207,  886.418,  717.576,  886.418, 1139.68 , 1181.89 ,
        1139.68 , 1097.47 , 1350.73 , 1392.94 , 1350.73 ,  717.576,
        1519.57 , 1392.94 , 1266.31 , 1435.15 , 1308.52 , 1139.68 ,
         844.207, 1097.47 , 1055.26 ,  970.838, 1055.26 ,  970.838,
         928.628,  801.997, 1055.26 ,  844.207, 1013.05 , 1139.68 ,
        1055.26 , 1055.26 , 1055.26 , 1477.36 , 1435.15 , 1308.52 ,
        1224.1  , 1308.52 , 1308.52 , 1477.36 , 